## CMS Data Cleaning - Step by Step Summary

### Problem
The raw CMS inpatient data contained hospital-level records for all DRG codes (diagnosis-related groups). To analyze treatment costs for our three diseases at the state level, we needed to:
- Filter to only Heart Failure, and COPD
- Aggregate from hospital level to state level
- Create weighted averages (so larger hospitals don't skew results)
- Organize by disease so we can compare costs across diseases


**Step 1: Load Data**
- Loaded 146,427 rows from CMS inpatient dataset
- 15 columns including provider info, DRG codes, and cost metrics

**Step 2: Filter to 2 Diseases**
- Kept only DRGs containing: HEART FAILURE, COPD/CHRONIC OBSTRUCTIVE
- Removed all other diagnoses
- Result: 5299 rows (2,962 Heart Failure + 2,337 COPD)

**Step 3: Create Disease Column**
- Created new "Disease" column by parsing DRG descriptions
- Mapped DRGs to diseases: Heart_Failure, COPD
- This allows us to track costs by disease type

**Step 4: Keep Only Required Columns**
- Kept: State, Disease, Total Discharges, Average Charges, Average Payment, Average Medicare Payment
- Removed: Hospital details, DRG codes, provider info (not needed for state-level analysis)

**Step 5: Remove Missing Values**
- Removed rows where payment data was null/empty
- Removed rows where covered charges, total payment, or Medicare payment was missing
- Result: 5299 rows (no data loss)

**Step 6: Remove Zero Discharge Rows**
- Removed rows where Total_Discharges = 0 (records with no patient volume)
- Result: 5299 rows (all rows had valid discharges)

**Step 7: Create Weighted Columns**
- Why weighted? Hospitals vary in size. A 100-bed hospital shouldn't have same weight as a 1,000-bed hospital.
- Formula: Weighted_Payment = Average_Payment × Total_Discharges
- Example: If cost is $10,000 per patient and hospital treated 500 patients, weighted = $5,000,000

**Step 8: Aggregate by State and Disease**
- Grouped all hospital records by state and disease
- Summed total discharges and weighted payments for each state-disease combination
- Result: 101 rows (51 states × 2 diseases = 102 combinations)

**Step 9: Calculate Weighted Averages**
- Formula: Weighted_Avg_Payment = Total_Weighted_Payment ÷ Total_Discharges
- This gives us the true average cost per patient at the state level, weighted by hospital size
- Example: If 50,000 patients total were treated across all hospitals in a state, this is the average cost per patient

**Step 10: Keep Only Final Columns**
- Kept: State, Disease, Total_Discharges, Avg_Total_Payment
- This is all we need for the state-disease level cost analysis

**Step 11: Pivot to Wide Format**
- Changed from long format (one row per state-disease) to wide format (one row per state)
- Created separate columns for each disease:
  - COPD_Cost, Heart_Failure_Cost
  - COPD_Discharges, Heart_Failure_Discharges
- Result: 51 rows (one per state)

**Step 12: Final Verification**
- Verified all values are reasonable
- Confirmed all 51 states have data for all three diseases


**Why Weighted Averages?**

Without weighting:
- Small hospital with 10 patients at $20,000/patient
- Large hospital with 1,000 patients at $10,000/patient
- Simple average: ($20,000 + $10,000) / 2 = $15,000 **which is wrong!**

With weighting:
- Small: (10 × $20,000) = $200,000
- Large: (1,000 × $10,000) = $10,000,000
- Weighted average: $10,200,000 / 1,010 patients = $10,099 **which is Correct!**

The weighted approach reflects reality: most patients (the 1,000) are treated at the large hospital at $10,000/patient.



In [1]:
# STEP 1: Load the data
import pandas as pd
cms = pd.read_csv("data/inpatient.csv", encoding="windows-1252", low_memory=False)

print(f"Total CMS rows: {len(cms)}")
print(f"\nColumns available:")
print(list(cms.columns))


Total CMS rows: 146427

Columns available:
['Rndrng_Prvdr_CCN', 'Rndrng_Prvdr_Org_Name', 'Rndrng_Prvdr_City', 'Rndrng_Prvdr_St', 'Rndrng_Prvdr_State_FIPS', 'Rndrng_Prvdr_Zip5', 'Rndrng_Prvdr_State_Abrvtn', 'Rndrng_Prvdr_RUCA', 'Rndrng_Prvdr_RUCA_Desc', 'DRG_Cd', 'DRG_Desc', 'Tot_Dschrgs', 'Avg_Submtd_Cvrd_Chrg', 'Avg_Tot_Pymt_Amt', 'Avg_Mdcr_Pymt_Amt']


In [2]:
# STEP 2: Keep only the 2 diseases we want

# Find DRGs related to our diseases
print("DRGs containing 'HEART FAILURE':")
hf_drgs = cms[cms['DRG_Desc'].str.contains('HEART FAILURE', case=False, na=False)]['DRG_Desc'].unique()
print(hf_drgs)
print(f"Count: {len(hf_drgs)}\n")

print("\nDRGs containing 'COPD' or 'CHRONIC OBSTRUCTIVE':")
copd_drgs = cms[cms['DRG_Desc'].str.contains('COPD|CHRONIC OBSTRUCTIVE', case=False, na=False)]['DRG_Desc'].unique()
print(copd_drgs)
print(f"Count: {len(copd_drgs)}\n")



print("\nTotal rows for each disease:")
hf_rows = len(cms[cms['DRG_Desc'].str.contains('HEART FAILURE', case=False, na=False)])
copd_rows = len(cms[cms['DRG_Desc'].str.contains('COPD|CHRONIC OBSTRUCTIVE', case=False, na=False)])

print(f"Heart Failure: {hf_rows} rows")
print(f"COPD: {copd_rows} rows")
print(f"Total: {hf_rows + copd_rows} rows")

DRGs containing 'HEART FAILURE':
['HEART FAILURE AND SHOCK WITH MCC' 'HEART FAILURE AND SHOCK WITH CC'
 'HEART FAILURE AND SHOCK WITHOUT CC/MCC']
Count: 3


DRGs containing 'COPD' or 'CHRONIC OBSTRUCTIVE':
['CHRONIC OBSTRUCTIVE PULMONARY DISEASE WITH MCC'
 'CHRONIC OBSTRUCTIVE PULMONARY DISEASE WITH CC'
 'CHRONIC OBSTRUCTIVE PULMONARY DISEASE WITHOUT CC/MCC']
Count: 3


Total rows for each disease:
Heart Failure: 2962 rows
COPD: 2337 rows
Total: 5299 rows


In [3]:
cms = cms[
    (cms["DRG_Desc"].str.contains("HEART FAILURE", case=False, na=False)) |
    (cms["DRG_Desc"].str.contains("COPD|CHRONIC OBSTRUCTIVE", case=False, na=False))
]
print(f"Rows after filtering to 2 diseases: {len(cms)}")


Rows after filtering to 2 diseases: 5299


In [4]:
# STEP 3: Create disease column
def get_disease(drg_desc):
    if "HEART FAILURE" in drg_desc.upper():
        return "Heart_Failure"
    elif "COPD" in drg_desc.upper() or "CHRONIC OBSTRUCTIVE" in drg_desc.upper():
        return "COPD"
    return None

cms["Disease"] = cms["DRG_Desc"].apply(get_disease)
print(f"Diseases created:")
print(cms["Disease"].value_counts())

Diseases created:
Disease
Heart_Failure    2962
COPD             2337
Name: count, dtype: int64


In [5]:
# STEP 4: Keep only required columns
cms = cms[[
    "Rndrng_Prvdr_State_Abrvtn",
    "Disease",
    "Tot_Dschrgs",
    "Avg_Tot_Pymt_Amt",
]].copy()

cms.rename(columns={
    "Rndrng_Prvdr_State_Abrvtn": "State",
    "Tot_Dschrgs": "Total_Discharges",
    "Avg_Tot_Pymt_Amt": "Avg_Total_Payment",
}, inplace=True)



In [6]:
# STEP 5: Remove missing values
print("\nSTEP 5: REMOVE MISSING VALUES")
print(f"Rows before: {len(cms)}")
cms = cms.dropna(subset=["Avg_Total_Payment"])
print(f"Rows after: {len(cms)}")

# STEP 6: Remove zero discharge rows
print("\nSTEP 6: REMOVE ZERO DISCHARGE ROWS")
print(f"Rows before: {len(cms)}")
cms = cms[cms["Total_Discharges"] > 0]
print(f"Rows after: {len(cms)}")


STEP 5: REMOVE MISSING VALUES
Rows before: 5299
Rows after: 5299

STEP 6: REMOVE ZERO DISCHARGE ROWS
Rows before: 5299
Rows after: 5299


In [7]:
# STEP 7: Create weighted columns
cms["Weighted_Total_Payment"] = cms["Avg_Total_Payment"] * cms["Total_Discharges"]

# STEP 8: Aggregate by State and Disease
print(f"Rows before aggregation: {len(cms)}")

cms_state = cms.groupby(["State", "Disease"], as_index=False).agg({
    "Total_Discharges": "sum",
    "Weighted_Total_Payment": "sum",
})

print(f"Rows after aggregation: {len(cms_state)}")


#  STEP 9: Calculate weighted average for Total Payment
cms_state["Avg_Total_Payment"] = cms_state["Weighted_Total_Payment"] / cms_state["Total_Discharges"]



Rows before aggregation: 5299
Rows after aggregation: 102


In [8]:

# STEP 10: Keep only final columns
cms_state = cms_state[[
    "State",
    "Disease",
    "Total_Discharges",
    "Avg_Total_Payment",
]]
cms_state.head()

,State,Disease,Total_Discharges,Avg_Total_Payment
0,AK,COPD,110,16610.600000
1,AK,Heart_Failure,590,16814.910169
2,AL,COPD,1355,8025.652399
3,AL,Heart_Failure,4548,9643.388742
4,AR,COPD,829,7521.352232


In [9]:
# STEP 11: Pivot to wide format
print(f"Rows before pivot: {len(cms_state)}")

# Pivot for costs
cms_pivot = cms_state.pivot_table(
    index="State",
    columns="Disease",
    values="Avg_Total_Payment",
    aggfunc="first"
).reset_index()
cms_pivot.columns.name = None

cms_pivot.rename(columns={
    "COPD": "COPD_Cost",
    "Heart_Failure": "Heart_Failure_Cost"
}, inplace=True)

print(f"Rows after pivot: {len(cms_pivot)}")


Rows before pivot: 102
Rows after pivot: 51


In [10]:
cms_pivot.head()

,State,COPD_Cost,Heart_Failure_Cost
0,AK,16610.600000,16814.910169
1,AL,8025.652399,9643.388742
2,AR,7521.352232,8912.169380
3,AZ,8773.520249,10827.994527
4,CA,11831.609250,14486.320625


In [13]:
# STEP 12: Final verification
print(f"Number of states: {len(cms_pivot)}")
print(f"\nAverage costs by disease across all states:")
print(f"Heart Failure: ${cms_state[cms_state['Disease']=='Heart_Failure']['Avg_Total_Payment'].mean():,.0f}")
print(f"COPD: ${cms_state[cms_state['Disease']=='COPD']['Avg_Total_Payment'].mean():,.0f}")


print(f"\nCost ranges by disease:")
print(f"Heart Failure: ${cms_state[cms_state['Disease']=='Heart_Failure']['Avg_Total_Payment'].min():,.0f} to ${cms_state[cms_state['Disease']=='Heart_Failure']['Avg_Total_Payment'].max():,.0f}")
print(f"COPD: ${cms_state[cms_state['Disease']=='COPD']['Avg_Total_Payment'].min():,.0f} to ${cms_state[cms_state['Disease']=='COPD']['Avg_Total_Payment'].max():,.0f}")


Number of states: 51

Average costs by disease across all states:
Heart Failure: $11,311
COPD: $9,542

Cost ranges by disease:
Heart Failure: $8,585 to $16,815
COPD: $7,419 to $16,611


In [14]:
cms_pivot.to_csv("data/clean_cms.csv", index=False)
